In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import datetime 
import math
import ephem
from datetime import date, timedelta
from numpy import unravel_index
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd
%config InlineBackend.figure_format = 'retina'

In [ ]:
def day_to_subsolar_point(days, time=1): # day = days after march 21, time = hours since 00:00:00
    
    UTCTime = time # hours since 00:00:00

    # equation for declination from: https://www.sciencedirect.com/topics/engineering/solar-declination
    δ=(23+27/60)*np.sin(np.deg2rad(360*days/365.25)) # declination on a particular day, = lat

    # references for equations: https://en.wikipedia.org/wiki/Equation_of_time
    # https://www.sciencedirect.com/science/article/pii/S0960148121004031?via%3Dihub#fd6
    B = (360) * (days - 81) / 365. # d = days since january 1
    EoT = (9.87 * np.sin(((2*B)*np.pi/180)) - 7.67 * np.sin((B + 78.7) * np.pi/180))
    lon = -15 * (UTCTime - 12 + EoT/60) 
            
    if lon < 0:
        lon = lon + 360

    return 90-δ, lon


def day_to_sublunar_point(days, time=1): # day = days after march 21
    date = days_since_march_21_to_month_day(days)
    day = date.day
    month = date.month
    
    greenwich = ephem.Observer()
    greenwich.lat = "0"
    greenwich.lon = "0"
    
    # make fromisoformat date string
    if month < 10: 
        m = "0" + str(month)
    else:
        m = str(month)
    if day < 10:
        d = "0" + str(day)
    else:
        d = str(day)
        
    if time < 10:
        t = "0" + str(time)
    else:
        t = str(time)
        
    # reference code from: https://stackoverflow.com/questions/52338971/confusion-with-using-dec-ra-to-compute-sub-lunar-location
    # code correctness 'checked' via: https://www.fourmilab.ch/cgi-bin/Earth, and https://www.mdpi.com/2072-4292/15/8/2151

    greenwich.date = datetime.datetime.fromisoformat('2023-'+m+'-'+d+'T'+t+':00:00')

    moon = ephem.Moon(greenwich)
    moon.compute(greenwich.date)
    moon_lon = math.degrees(moon.ra - greenwich.sidereal_time() )
    # map longitude value from -180 to +180 
    moon_lon = moon_lon + 180
    if moon_lon > 360:
        moon_lon = moon_lon - 360
    if moon_lon < 0:
        moon_lon = 360 - abs(moon_lon)

    moon_lat = math.degrees(moon.dec)
    if moon_lat > 0: 
        moon_lat = 90 - moon_lat
    if moon_lat < 0: 
        moon_lat = 90 + abs(moon_lat)
            
    return moon_lat, moon_lon


def days_since_march_21_to_month_day(days):
   # https://stackoverflow.com/questions/38691545/python-convert-days-since-1990-to-datetime-object
    start = date(2023,3,21)
    delta = timedelta(days)
    offset = start + delta
    return offset

In [ ]:
R_earth = 6378 * 1000 # m
M_sun = 1988500e24
M_moon = 0.07346e24
M_earth = 5.9722e24
# approximation: assuming orbits are circular
R_earth_moon = 384400 * 1000 # m
R_earth_sun = 150 * 1000000 * 1000 # m

# reference for force calculations: https://www.cambridge.org/us/files/1913/6681/8626/7708_Tidal_distortion.pdf

def F_g(M,m,R,G=6.67384e-11):
    # G: in m^3/(kg*s^2)
    return G*(M*m)/(R**2) # in inwards radial direction

def lat_long_to_spherical(lat,long,p):
    # convert lat, long to xyz coordinates (using spherical transformation), center of earth is at 0,0,0
    # in degrees, 90º lat is north pole, -90º lat is south pole
    theta = long
    phi = lat
    x = p*np.sin(np.deg2rad(phi))*np.cos(np.deg2rad(theta))
    y = p*np.sin(np.deg2rad(phi))*np.sin(np.deg2rad(theta))
    z = p*np.cos(np.deg2rad(phi))
    return x,y,z

# centrifugal force on earth due to rotation around CM of earth-moon system
def F_centrifugal2(m, xm,ym,zm, x,y,z, G=6.67384e-11): # returns a vector, should be perpendicular to axis of rotation
    # F = ma for rotational: F = mv^2/r
    # F = ma, F = m*(omega**2) * s, s = distance from rotation axis 
    # w around CM of earth-moon system
    # https://www.cambridge.org/us/files/1913/6681/8626/7708_Tidal_distortion.pdf
    w = (G * (M_earth + M_moon) / ((R_earth_moon + R_earth)**3))
    CM = (M_moon / (M_earth+M_moon)) * (R_earth_moon + R_earth)
    Fc = -m * w**2 * CM
    
    Fc2 = G * 1 * M_moon / ((R_earth_moon + R_earth)**2)
    
    # magnitude & direction of force is the same for any point on earth, opposes moon gravitational force   
    
    #calculate vector: directed away from barycenter (opposite direction as moon gravity)
    to_moon_unit_vector = np.array([-xm,-ym,-zm]) / ((xm**2 + ym**2 + zm**2)**0.5) # direction
    Fc_vector = Fc2 * to_moon_unit_vector
        
    return Fc_vector

# centrifugal force on earth due to rotation around CM of earth-sun system
def F_centrifugal3(m, xs,ys,zs, x,y,z, G=6.67384e-11): # returns a vector, should be perpendicular to axis of rotation
    # F = ma for rotational: F = mv^2/r
    # F = ma, F = m*(omega**2) * s, s = distance from rotation axis 
    # w around CM of earth-sun system
    # https://www.cambridge.org/us/files/1913/6681/8626/7708_Tidal_distortion.pdf
    w = (G * (M_earth + M_sun) / ((R_earth_sun + R_earth)**3))
    CM = (M_sun / (M_earth+M_sun)) * (R_earth_sun + R_earth)
    Fc = -m * w**2 * CM
    
    Fc2 = G * 1 * M_sun / ((R_earth_sun + R_earth)**2)
    
    # magnitude & direction of force is the same for any point on earth, opposes sun gravitational force   
    
    #calculate vector: directed away from barycenter (opposite direction as sun gravity)
    to_sun_unit_vector = np.array([-xs,-ys,-zs]) / ((xs**2 + ys**2 + zs**2)**0.5) # direction
    Fc_vector = Fc2 * to_sun_unit_vector
        
    return Fc_vector

In [ ]:
def calc_moon_grav(xm,ym,zm,x,y,z,G=6.67384e-11): # calculate moon gravity effect on any point on earth's surface for a given sublunar point
    distance_to_moon = (sum((np.array([x,y,z]) - np.array([xm,ym,zm]))**2))**0.5 # euclidean distance
            
    Fmoon = F_g(M_moon, 1, distance_to_moon)
            
    # calculate vector
    unit_vector = np.array([xm-x,ym-y,zm-z]) / (distance_to_moon)
    vector = Fmoon * unit_vector
        
    return vector
    
def calc_sun_grav(xs,ys,zs,x,y,z,G=6.67384e-11):# calculate sun gravity effect on any point on earth's surface for a given subsolar point
    distance_to_sun = (sum((np.array([x,y,z]) - np.array([xs,ys,zs]))**2))**0.5 # euclidean distance
    
    Fsun = F_g(M_sun, 1, distance_to_sun)
    
    # calculate vector
    unit_vector = np.array([xs-x,ys-y,zs-z]) / (distance_to_sun)
    vector = Fsun * unit_vector
        
    return vector

In [ ]:
def calc_tide_type(lat_sublunar, long_sublunar, lat_subsolar, long_subsolar): # get angles between moon and sun with respect to 0,0,0 (center of earth)
    
    # reference: https://oceanservice.noaa.gov/facts/springtide.html
    # spring tide: moon and sun are nearly in alignment (happens during full and new moon)
    # neap tide: moon and sun are at right angles (happens during half moons) (~7 days after spring tide)
    
    xm,ym,zm = lat_long_to_spherical(lat_sublunar,long_sublunar,R_earth)
    xs,ys,zs = lat_long_to_spherical(lat_subsolar,long_subsolar,R_earth)
    
    # dot product: a . b = |a||b| cos(theta)
    # cos(theta) = (a . b) / (|a||b|)
    # theta = cos-1((a . b) / (|a||b|))
    
    dot = xm * xs + ym * ys + zm * zs
    mag_m = (xm**2 + ym**2 + zm**2)**0.5 
    mag_s = (xs**2 + ys**2 + zs**2)**0.5 
    theta = np.rad2deg(np.arccos(dot / (mag_m * mag_s)))
    
    return theta

In [ ]:
def calc_net_F_moon_and_sun(lat_sublunar, long_sublunar, lat_subsolar, long_subsolar, points): # calculate total net force on a 2d array of points given lists of sublunar and subsolar points
    
    # convert lat and long to xyz coordinates
    xm,ym,zm = lat_long_to_spherical(lat_sublunar,long_sublunar,R_earth+R_earth_moon)
    xs,ys,zs = lat_long_to_spherical(lat_subsolar,long_subsolar,R_earth+R_earth_sun)
    
    for i in range(points.shape[0]):
        for j in range(points.shape[1]):
            lat = i
            long = j
            x,y,z = lat_long_to_spherical(lat,long,R_earth)
            
            # moon
            
            # calculate moon gravity at that point
            m_vector = calc_moon_grav(xm,ym,zm,x,y,z)
            
            # calculate centrifugal force 
            m_v_cent = F_centrifugal2(1, xm,ym,zm,x,y,z)

            m_sum_vectors = m_vector + m_v_cent 
            
            
            # sun
            
            # calculate sun gravity at that point
            s_vector = calc_sun_grav(xs,ys,zs,x,y,z)
            
            # calculate centrifugal force 
            s_v_cent = F_centrifugal3(1, xs,ys,zs,x,y,z)

            s_sum_vectors = s_vector + s_v_cent 
            
            total_sum = m_sum_vectors + s_sum_vectors
            
            F = sum(total_sum**2)**0.5
            
            points[i][j] = F
        
    return points


In [ ]:
%matplotlib inline
n = 360
points = np.zeros((n//2,n))

las, los = day_to_subsolar_point(0)
lam, lom = day_to_sublunar_point(0) 

plt.figure()
m_points = calc_net_F_moon_and_sun(lam,lom,las,los,points)
img = plt.imshow(points, cmap='YlGnBu_r')
plt.colorbar()
plt.scatter(lom,lam)
plt.scatter(los,las)
plt.show()


In [ ]:
%matplotlib tk

In [ ]:
from matplotlib.animation import FFMpegWriter
metadata = dict(title='My first animation in 2D', artist='Matplotlib',comment='Wakanda is coming.')
writer = FFMpegWriter(fps=15, metadata=metadata)

fig = plt.figure(figsize=(10,8))

n = 360
points = np.zeros((n//2,n))
las_pts = []
los_pts = []
lam_pts = []
lom_pts = []
back = 80
for i in range(-back,365-back):
    las, los = day_to_subsolar_point(i)
    lam, lom = day_to_sublunar_point(i) 
    if lom > 359:
        lom = 359 # for display purposes
    las_pts.append(las)
    los_pts.append(los)
    lam_pts.append(lam)
    lom_pts.append(lom)
    
# split up moon coordinates to avoid ugly plotting due to wraps
arrs = []
arrs2 = []
arr = lom_pts.copy()
arr2 = lam_pts.copy()
index = 0
i = 0
for i in range(len(arr)-1):
    if arr[i] > arr[i+1]: # wrapped
        arrs.append(arr[index:i+1])
        arrs2.append(arr2[index:i+1])
        index = i+1
    i+=1   

index = 0
index_in_subarrays = 0

thetas = []

data = np.loadtxt('topography_180x360_grid.txt')

maxvals_in_points = []


with writer.saving(fig, "animation3.mp4", dpi=200):
    
    for i in range(len(lam_pts)-1):
        points = calc_net_F_moon_and_sun(lam_pts[i], lom_pts[i], las_pts[i], los_pts[i], points)
        theta = round(calc_tide_type(lam_pts[i], lom_pts[i], las_pts[i], los_pts[i]), 0)
        
        
        ind = unravel_index(points.argmax(), points.shape)
        maxvals_in_points.append(points[ind[0], ind[1]])

        ax1 = fig.add_subplot()
        
        ax1.imshow(data>0, cmap="gray", alpha=1)
        
        img = ax1.imshow(points, cmap='YlGnBu_r', clim=(0,1.6e-6), alpha = 0.96)

        
        if lom_pts[i] > lom_pts[i+1]:
            index_in_subarrays += 1
            index = i+1
            
        for j in range(0, index_in_subarrays):
            ax1.plot(arrs[j], arrs2[j], c='white', alpha = 0.3)
        
        ax1.plot(lom_pts[index:i+1],lam_pts[index:i+1], c='white', alpha=0.3)
        ax1.scatter(lom_pts[i],lam_pts[i], label='sublunar point')
        ax1.plot(los_pts[0:i+1],las_pts[0:i+1], c='yellow')
        ax1.scatter(los_pts[i],las_pts[i], label='subsolar point')
        ax1.set_xlabel("east longitude")
        ax1.set_ylabel("colatitude")
        
        ax1.plot(np.linspace(0,359,3), np.linspace(90,90,3), 'r--', alpha=0.25, label='equator')

        ax1.set_title(r"$\bf{" + str(days_since_march_21_to_month_day(i-back)) + "}$" + "\n" + "net force due to Sun gravity + Moon gravity + centrifugal forces")
        plt.legend(fontsize=8)
        
        # subplot arranging code from https://stackoverflow.com/questions/44682146/align-subplot-with-colorbar
        divider = make_axes_locatable(ax1)
        ax2 = divider.append_axes("bottom", size="50%", pad=0.8)
        cax = divider.append_axes("right", size="5%", pad=0.08)
        cb = plt.colorbar(img, ax=ax1, cax=cax, label='F magnitude')
        
        thetas.append(theta)
        ax2.plot(thetas, maxvals_in_points, alpha = 0.5)
        ax2.scatter(thetas[-1], maxvals_in_points[-1])
        ax2.set_ylim(0.5e-6,1.75e-6)
        ax2.set_xlim(0,180)
        ax2.set_title("angle $\dot{\Theta}$ between Sun and Moon vs max net F")
        ax2.set_xlabel("angle $\dot{\Theta}$ (relative to center of Earth)")
        ax2.set_ylabel("max net F")

        plt.show()
        plt.draw()
        plt.pause(0.01)
        writer.grab_frame()
        
        cb.remove()
        ax1.clear()
        ax2.clear()
        fig.clear()


In [ ]:
# make first animation
# read in NOAA data
water_levels_noaa = pd.read_csv('CO_OPS_8411060_wl.csv')
noaa_x = water_levels_noaa['Time (GMT)']
noaa_x = [int(i[0:2]) + int(i[3:5])/60 for i in noaa_x] # strip off :00 and make an integer instead of a string
noaa_y = water_levels_noaa['Verified (ft)']

In [ ]:
# calculate sun / moon / Fc 
n = 360
points = np.zeros((n//2,n))
las_pts = []
los_pts = []
lam_pts = []
lom_pts = []
day = 14
for i in range(0, 24):
    las, los = day_to_subsolar_point(day, i)
    lam, lom = day_to_sublunar_point(day, i) 
    if lom > 359:
        lom = 359 # for display purposes
    las_pts.append(las)
    los_pts.append(los)
    lam_pts.append(lam)
    lom_pts.append(lom)
# get the last hour
day = day + 1
i = 0
las, los = day_to_subsolar_point(day, i)
lam, lom = day_to_sublunar_point(day, i) 
if lom > 359:
    lom = 359 # for display purposes
las_pts.append(las)
los_pts.append(los)
lam_pts.append(lam)
lom_pts.append(lom)
    
# split up moon coordinates to avoid ugly plotting due to wraps
arrs = []
arrs2 = []
arr = lom_pts.copy()
arr2 = lam_pts.copy()
index = 0
i = 0
for i in range(len(arr)-1):
    if arr[i] < arr[i+1]: # wrapped
        arrs.append(arr[index:i+1])
        arrs2.append(arr2[index:i+1])
        index = i+1  

index = 0
index_in_subarrays = 0

# split up sun coordinates to avoid ugly plotting due to wraps
arrss = []
arrs2s = []
arr_s = los_pts.copy()
arr2_s = las_pts.copy()
indexs = 0
i_s = 0
for i in range(len(arr)-1):
    if arr_s[i] < arr_s[i+1]: # wrapped
        arrss.append(arr_s[indexs:i+1])
        arrs2s.append(arr2_s[indexs:i+1])
        indexs = i+1

indexs = 0
index_in_subarrayss = 0


thetas = []

data = np.loadtxt('topography_180x360_grid.txt')

maxvals_in_points = []


In [ ]:
from matplotlib.animation import FFMpegWriter
metadata = dict(title='My first animation in 2D', artist='Matplotlib',comment='Wakanda is coming.')
writer = FFMpegWriter(fps=15, metadata=metadata)

fig = plt.figure(figsize=(10,8))

with writer.saving(fig, "animation.mp4", dpi=200):
    
    for i in range(len(lam_pts)):
        points = calc_net_F_moon_and_sun(lam_pts[i], lom_pts[i], las_pts[i], los_pts[i], points)
        theta = round(calc_tide_type(lam_pts[i], lom_pts[i], las_pts[i], los_pts[i]), 0)
        
        
        ind = unravel_index(points.argmax(), points.shape)
        maxvals_in_points.append(points[ind[0], ind[1]])

        ax1 = fig.add_subplot()
        
        ax1.imshow(data>0, cmap="gray", alpha=1)
        
        img = ax1.imshow(points, cmap='YlGnBu_r', clim=(0,1.6e-6), alpha = 0.96)

        if i+1 < len(lam_pts):
            if lom_pts[i] < lom_pts[i+1]:
                index_in_subarrays += 1
                index = i+1

            if los_pts[i] < los_pts[i+1]:
                index_in_subarrayss += 1
                indexs = i+1

        for j in range(0, index_in_subarrays):
            ax1.plot(arrs[j], arrs2[j], c='white', alpha = 0.5)

        for j in range(0, index_in_subarrayss):
            ax1.plot(arrss[j], arrs2s[j], c='yellow', alpha = 0.5)
        
        ax1.plot(lom_pts[index:i+1],lam_pts[index:i+1], c='white', alpha=0.5)
        ax1.scatter(lom_pts[i],lam_pts[i], label='sublunar point')
        ax1.plot(los_pts[indexs:i+1],las_pts[indexs:i+1], c='yellow', alpha = 0.5)
        ax1.scatter(los_pts[i],las_pts[i], label='subsolar point')
        ax1.set_xlabel("east longitude")
        ax1.set_ylabel("colatitude")
        
        ax1.scatter([360-67], [90-44], marker='*', color='yellow', label='NOAA station 8411060')
        
        ax1.plot(np.linspace(0,359,3), np.linspace(90,90,3), 'r--', alpha=0.25, label='equator')
        
        plt.legend(fontsize=8, loc='upper left')

        
        if i < 10:
            t = "0" + str(i)
        else:
            t = str(i)
        ax1.set_title(r"$\bf{" + str(days_since_march_21_to_month_day(day)) + "}$" + "\n" + r"$\bf{" + " UTC= " + t + ":00:00 hrs"+ "}$" + "\n" + "net force due to Sun gravity + Moon gravity + centrifugal forces")
        
        
        # subplot arranging code from https://stackoverflow.com/questions/44682146/align-subplot-with-colorbar
        divider = make_axes_locatable(ax1)

        ax2 = divider.append_axes("bottom", size="50%", pad=0.8)
        cax = divider.append_axes("right", size="5%", pad=0.08)
        cb = plt.colorbar(img, ax=ax1, cax=cax, label='F magnitude')
        

        thetas.append(theta)
        
        if i == 0:
            ax2.plot(noaa_x[0:(i*10)], noaa_y[0:(i*10)], alpha = 0.75)
            ax2.scatter(noaa_x[i*10], noaa_y[i*10])
        else: 
            ax2.plot(noaa_x[0:(i*10-1)], noaa_y[0:(i*10-1)], alpha = 0.75)
            ax2.scatter(noaa_x[i*10-1], noaa_y[i*10-1])
        ax2.set_ylim(-1,18)
        ax2.set_xlim(0,24)
        ax2.set_title("water levels at NOAA station 8411060 (Cutler Farris Wharf ME)")
        ax2.set_xlabel("time (hrs, UTC)")
        ax2.set_ylabel("water level (feet)")
        
                

        plt.show()
        plt.draw()
        plt.pause(0.05)
        writer.grab_frame()
        
        cb.remove()
        ax1.clear()
        ax2.clear()
        fig.clear()
